In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Whole_analysis').getOrCreate()

#TO MODIFY!!!

In [2]:
df = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/price_elasticity_model_data/part-*',
    header=True)


In [3]:
df.createOrReplaceTempView('model_data_table')


In [4]:
#df.show()

In [5]:
df_d_item = spark.read.parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_item/*')


In [6]:
df_d_item.createOrReplaceTempView('d_item_tmp')

In [7]:
df_item_visits = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/item_visits/*202*/*.parquet',header=True)

In [8]:
df_f_orders = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/f_orders/*.parquet',header=True)

df_f_orders.createOrReplaceTempView('f_orders')

In [9]:
df_d_calendar = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_calendar/*.parquet',header=True)

df_d_calendar.createOrReplaceTempView('d_calendar')



In [10]:
df_d_order_flags = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_order_flags/*.parquet',header=True)

df_d_order_flags.createOrReplaceTempView('d_order_flags')



In [11]:
df_list_top_seller = spark.sql("""
SELECT di.item_code,
       sum(fo.amount + amount_discount) AS order_amount

  FROM f_orders as fo
       INNER JOIN d_calendar c ON c.date_skey = fo.order_date_skey
       INNER JOIN d_item_tmp di ON di.item_skey = fo.item_skey
INNER JOIN d_order_flags dof on dof.order_flags_skey = fo.order_flags_skey

 WHERE c.date >= '2020-01-01' --current_date - 1*interval ' 1 year'
and dof.is_cancelled_immediately = 'is not cancelled immediately'
and dof.is_cancelled_after_payment = 'is not cancelled after payment'
and dof.is_cancelled_before_payment = 'is not cancelled before payment'
group by di.item_code
ORDER BY order_amount DESC""")

In [12]:
df_list_top_seller.createOrReplaceTempView('top_seller_list')

#df_list_top_seller.show()



In [13]:
df_list_top_seller_all= spark.sql("""
SELECT item_code
from top_seller_list""")



In [14]:
top_seller_list_string_all = df_list_top_seller_all.rdd.map(lambda x: x.item_code).collect()


In [15]:
import numpy as np

N_items = len(top_seller_list_string_all)
print(N_items)

map_of_items = {}


94750


In [16]:
#DEFINE Number of items and get list
print(top_seller_list_string_all)

['000000001000021540', '000000001000021539', '000000001000016021', '000000001000054983', '000000001000054984', '000000001000021718', '000000001000016133', '000000001000062031', '000000001000113432', '000000001000015496', '000000001000015484', '000000001000073170', '000000001000021719', '000000001000015488', '000000001000121274', '000000001000055017', '000000001000008754', '000000001000055018', '000000001000021533', '000000001000008809', '000000001000062037', '000000001000021542', '000000001000008736', '000000001000015958', '000000001000062032', '000000001000127680', '000000001000021541', '000000001000117189', '000000000000001213', '000000000000001225', '000000001000021534', '000000001000032622', '000000001000016020', '000000001000032236', '000000001000021698', '000000001000015723', '000000001000085415', '000000001000086500', '000000001000005892', '000000001000016019', '000000001000008776', '000000001000032242', '000000001000091266', '000000001000056742', '000000001000008546', '00000000

In [17]:
def get_list_of_items(top_seller_list = top_seller_list_string_all):
    selected_items = top_seller_list
    selected_items[0]= "'" + selected_items[0]
    selected_items[-1]= selected_items[-1] + "'"
    separator =  "' , '"
    f_string = separator.join(selected_items)
    return f_string



final_string = get_list_of_items()


In [18]:
df_item_visits.createOrReplaceTempView('item_visits')

df_item_visits_df = spark.sql("""SELECT item_code,
                                         meta_date,
                                        SUM(unique_views) as all_unique_views
                                    from item_visits
                                  where item_code in ({lista})
                                  GROUP BY item_code, meta_date""".format(lista=final_string))
#df_item_visits_df.show()

In [19]:
df2 = spark.sql("""SELECT * from model_data_table where update_date >= '2020-01-01' and item_code in ({lista}) ORDER BY update_date""".format(lista=final_string))
#df2.show()

In [20]:
df4 = spark.sql ("""SELECT item_code, item_main_category, item_sub_category_1, item_sub_category_2,item_parent_item_code from d_item_tmp where item_code in ({lista})""".format(lista=final_string))
#df4.show()

In [21]:
df4.createOrReplaceTempView('d_item_filtered')
df2.createOrReplaceTempView('model_data_table_YTD')
df_item_visits_df.createOrReplaceTempView('item_views_YTD')

In [22]:
spark.sql("""SELECT COUNT(DISTINCT item_code) from item_views_YTD""").show()
#df_item_visits_df.describe(['item_code']).show()

+-------------------------+
|count(DISTINCT item_code)|
+-------------------------+
|                    94663|
+-------------------------+



In [23]:
sql_price_visit_join= """ SELECT mdt_YTD.*, it_YTD.all_unique_views FROM model_data_table_YTD as mdt_YTD LEFT JOIN item_views_YTD as it_YTD
on mdt_YTD.item_code = it_YTD.item_code and mdt_YTD.update_date = it_YTD.meta_date """

print(sql_price_visit_join)

 SELECT mdt_YTD.*, it_YTD.all_unique_views FROM model_data_table_YTD as mdt_YTD LEFT JOIN item_views_YTD as it_YTD
on mdt_YTD.item_code = it_YTD.item_code and mdt_YTD.update_date = it_YTD.meta_date 


In [24]:
df_price_visit_join = spark.sql(sql_price_visit_join)
df_price_visit_join.createOrReplaceTempView('mdt_YTD_it')

#df_price_visit_join.describe(['item_code']).show()
#df_price_visit_join.show()

In [25]:
sql_top_item_query_group_by = """
WITH tmp_tbl AS (
      SELECT *,
             CASE
                 WHEN lag(delivery_weeks, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      delivery_weeks
                     AND
                      lag(item_price, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      item_price
                     THEN NULL
                 ELSE RANK() OVER (PARTITION BY item_code ORDER BY update_date)
                 END AS ranking_col --filled with row number or delivery week if the previous one is part of the same group,
        FROM mdt_YTD_it
  ),
       tmp_tbl2 AS (
           SELECT update_date,
                  item_code,
                  sales,
                  all_unique_views,
                  item_price,
                  delivery_weeks,
                  CASE
                      WHEN ranking_col IS NULL
                          THEN last(ranking_col, True) OVER (PARTITION BY item_code ORDER BY update_date ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING)
                      ELSE ranking_col
                      END AS ranks
             FROM tmp_tbl
       )
SELECT CAST(MIN(update_date) as date)                     AS min_date,
       CAST (MAX(update_date) as date)                    AS max_date,
       item_code,
       item_price,
       delivery_weeks,
       avg(sales) as avg_sales,
       avg(all_unique_views) as avg_unique_views,
       CASE WHEN sum(all_unique_views) <> 0 THEN sum(sales)/sum(all_unique_views) ELSE NULL END as conv_rate,
       CAST (datediff(MAX(update_date), MIN(update_date) ) + 1 as int) AS bin
  FROM tmp_tbl2
 GROUP BY ranks, delivery_weeks, item_price, item_code
 ORDER BY item_code, min_date ASC;
"""

In [26]:
print(sql_top_item_query_group_by)


WITH tmp_tbl AS (
      SELECT *,
             CASE
                 WHEN lag(delivery_weeks, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      delivery_weeks
                     AND
                      lag(item_price, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      item_price
                     THEN NULL
                 ELSE RANK() OVER (PARTITION BY item_code ORDER BY update_date)
                 END AS ranking_col --filled with row number or delivery week if the previous one is part of the same group,
        FROM mdt_YTD_it
  ),
       tmp_tbl2 AS (
           SELECT update_date,
                  item_code,
                  sales,
                  all_unique_views,
                  item_price,
                  delivery_weeks,
                  CASE
                      WHEN ranking_col IS NULL
                          THEN last(ranking_col, True) OVER (PARTITION BY item_code ORDER BY update_date ROWS B

In [27]:
sql_top_item_df = spark.sql(sql_top_item_query_group_by)

In [28]:
#sql_top_item_df.show(5000,False)

In [29]:
#CREATE A TABLE VIEW FOR Top10 sellers
sql_top_item_df.createOrReplaceTempView('top_sellers')


In [30]:
#JOIN THE TWO TABLEs
sql_join_query = " SELECT top.*, di.item_parent_item_code, di.item_main_category, di.item_sub_category_1, di.item_sub_category_2" \
                 "        from top_sellers as top " \
                 "INNER JOIN d_item_filtered as di on di.item_code = top.item_code "\
                 "ORDER BY top.min_date ASC"

In [31]:
print(sql_join_query)


 SELECT top.*, di.item_parent_item_code, di.item_main_category, di.item_sub_category_1, di.item_sub_category_2        from top_sellers as top INNER JOIN d_item_filtered as di on di.item_code = top.item_code ORDER BY top.min_date ASC


In [32]:
final_top_seller_df = spark.sql(sql_join_query)

In [33]:
#final_top_seller_df.show(3000,False)


In [34]:
#final_top_seller_df.describe(['bin']).show()

In [35]:
pd_df_top_sellers = final_top_seller_df.toPandas()


In [36]:
import pickle
import pandas as pd

In [37]:
pd_df_top_sellers['log_price'] = np.log(pd_df_top_sellers['item_price']+0.0001)

In [38]:
pd_df_top_sellers['log_delivery_weeks'] = np.log(pd_df_top_sellers['delivery_weeks']+0.0001)

/Users/gabriele.sabato/anaconda3/envs/pythonProject/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [39]:
pd_df_top_sellers['log_sales'] = np.log(pd_df_top_sellers['avg_sales']+0.0001)


In [40]:
pd_df_top_sellers['log_avg_unique_views'] = np.log(pd_df_top_sellers['avg_unique_views']+0.0001)


In [41]:
pd_df_top_sellers['log_conv_rate']= np.log(pd_df_top_sellers['conv_rate']+0.0001)
pd_df_top_sellers['conv_rate']= pd_df_top_sellers['conv_rate']+0.000001

In [42]:
pd_df_top_sellers.to_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/All_catalogue_seller_YTD_group_by_item_visit_20210226.pickle')